<a href="https://colab.research.google.com/github/A5inity/Machine-Learning-Basics/blob/main/Binary_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import tf_keras
import pandas as pd
from tensorflow_hub.keras_layer import KerasLayer
from google.colab import drive
drive.mount('/content/drive')
import tensorflow_datasets as tfds

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_data, test_data = tfds.load('imdb_reviews', split=["train", "test"], batch_size=-1, as_supervised=True)
train_examples, train_labels=tfds.as_numpy(train_data)
test_examples, test_labels=tfds.as_numpy(test_data)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.GTPO5D_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.GTPO5D_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.GTPO5D_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))


Training entries: 25000, test entries: 25000


In [ ]:
import tensorflow_hub as hub
from tensorflow.keras import layers
model = "https://tfhub.dev/google/nnlm-en-dim128/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
import tf_keras
from tensorflow_hub.keras_layer import KerasLayer
model_url = "https://tfhub.dev/google/nnlm-en-dim50/2" # Use a different variable name for the model URL
hub_layer = hub.KerasLayer(model_url, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

model=tf_keras.Sequential() # Use 'model' for the Keras model
model.add(hub_layer)
model.add(tf_keras.layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_1 (Dense)             (None, 1)                 51        
                                                                 
Total params: 48190651 (183.83 MB)
Trainable params: 48190651 (183.83 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import os
os.environ["TF_USE_LEGACY_KERAS"] ="1"

In [ ]:
Xax=train_examples[0:10000]

Yax=train_labels[0:10000]

Ypartial=train_labels[10000:]
Xpartial=train_examples[10000:]
history=model.fit(Xax, Yax, epochs=20, batch_size=1024, validation_data=(Xpartial, Ypartial), verbose=1)

Epoch 1/20
10/10 [==============================] - 19s 2s/step - loss: 0.6974 - accuracy: 0.5301 - val_loss: 0.6729 - val_accuracy: 0.5891
Epoch 2/20
10/10 [==============================] - 18s 2s/step - loss: 0.6468 - accuracy: 0.6699 - val_loss: 0.6361 - val_accuracy: 0.7089
Epoch 3/20
10/10 [==============================] - 20s 2s/step - loss: 0.6042 - accuracy: 0.7773 - val_loss: 0.6055 - val_accuracy: 0.7501
Epoch 4/20
10/10 [==============================] - 20s 2s/step - loss: 0.5649 - accuracy: 0.8218 - val_loss: 0.5767 - val_accuracy: 0.7763
Epoch 5/20
10/10 [==============================] - 20s 2s/step - loss: 0.5286 - accuracy: 0.8443 - val_loss: 0.5502 - val_accuracy: 0.7943
Epoch 6/20
10/10 [==============================] - 20s 2s/step - loss: 0.4937 - accuracy: 0.8619 - val_loss: 0.5252 - val_accuracy: 0.8042
Epoch 7/20
10/10 [==============================] - 20s 2s/step - loss: 0.4601 - accuracy: 0.8768 - val_loss: 0.5018 - val_accuracy: 0.8134
Epoch 8/20
10/10 [==

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
results = model.evaluate(test_examples, test_labels)
print(results)

#model.save('/content/drive/My Drive/IMDB Review Model/IMDBREV_Model_Aashir.keras')

782/782 [==============================] - 148s 189ms/step - loss: 0.3578 - accuracy: 0.8509
[0.3577899932861328, 0.8509200215339661]


In [ ]:
def predict_sentiment(review, model):
    prediction = model.predict([review])[0]
    sentiment = 'Positive' if prediction >= 0.5 else 'Negative'  # Threshold at 0.5 for sigmoid
    return sentiment



new_review_1 = "The plot was dull and uninspiring."
new_review_2 = "Wow, never knew I could sleep so fast!"
new_review_3 = "A family film in every sense and one that deserves the praise it received."
new_review_4 = "Really Jodie Whittaker? Did all the other actors in the World Die?"
new_review_4 = "Das ist völliger Blödsinn, ich habe meine Zeit verschwendet." # Translation : That is complete nonsense, I have wasted my time
new_review_5= "Finally! a good movie!!"
print(f"Review: '{new_review_1}' -> Prediction: {predict_sentiment(new_review_1, model)}")
print(f"Review: '{new_review_2}' -> Prediction: {predict_sentiment(new_review_2, model)}")
print(f"Review: '{new_review_3}' -> Prediction: {predict_sentiment(new_review_3, model)}")
print(f"Review: '{new_review_4}' -> Prediction: {predict_sentiment(new_review_4, model)}")
print(f"Review: '{new_review_5}' -> Prediction: {predict_sentiment(new_review_5, model)}")

1/1 [==============================] - 0s 471ms/step
Review: 'The plot was dull and uninspiring.' -> Prediction: Negative
1/1 [==============================] - 0s 249ms/step
Review: 'Wow, never knew I could sleep so fast!' -> Prediction: Negative
1/1 [==============================] - 0s 205ms/step
Review: 'A family film in every sense and one that deserves the praise it received.' -> Prediction: Positive
1/1 [==============================] - 0s 213ms/step
Review: 'Das ist völliger Blödsinn, ich habe meine Zeit verschwendet.' -> Prediction: Negative
1/1 [==============================] - 0s 221ms/step
Review: 'Finally! a good movie!!' -> Prediction: Positive


In [ ]:
train_data, test_data = tfds.load('imdb_reviews', split=["train", "test"], batch_size=-1, as_supervised=True)
train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

df = pd.DataFrame({'review': test_examples, 'actual_label': test_labels})


probabilities = model.predict(test_examples)


df['predicted_probability'] = probabilities


df['predicted_sentiment'] = np.where(df['predicted_probability'] >= 0.5, 'Positive', 'Negative')


print(df.head(20))

782/782 [==============================] - 141s 180ms/step
                                               review  actual_label  \
0   b"There are films that make careers. For Georg...             1   
1   b"A blackly comic tale of a down-trodden pries...             1   
2   b'Scary Movie 1-4, Epic Movie, Date Movie, Mee...             0   
3   b'Poor Shirley MacLaine tries hard to lend som...             0   
4   b'As a former Erasmus student I enjoyed this f...             1   
5   b"My God, Ryan Gosling has made a lot of deep ...             1   
6   b"This film just won the best film award at th...             1   
7   b'The cast for this production of Rigoletto is...             1   
8   b'As long as you keep in mind that the product...             0   
9   b"Every great once in a while, you stumble upo...             1   
10  b'I\'ll give it a two because it has a lot of ...             0   
11  b"I kind of like Bam Margera, so I was curious...             0   
12  b'The plot lin